In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from itertools import accumulate
from IPython.display import Image

import geopandas as gpd
import dask_geopandas as dask_gpd
import dask
from dask import delayed, compute
from dask.dataframe import dd
from dask.diagnostics import ProgressBar
from dask.distributed import Client, LocalCluster

import time
import sys

ImportError: cannot import name 'dd' from 'dask.dataframe' (/Users/julisn/anaconda3/lib/python3.11/site-packages/dask/dataframe/__init__.py)

### Tree Class, used to organize overlapping lidar_points and corresponding tree crowns:

In [ ]:
class Tree:
    def __init__(self, tree_id, lidar_points):
        self.tree_id = tree_id
        self.lidar_points = lidar_points
        
    def __str__(self):
        return f"Tree_ID: {self.tree_id}, Lidar_Points: {self.lidar_points}"

### Import the voxel metrics data generated by the R script:

In [ ]:
voxel_metrics = pd.read_csv(r"R files/R file outputs/SB2_trees_only_buff.csv")
voxel_df = voxel_metrics.sort_values(by=['X', 'Y'])

### Importing tree crown data:

In [ ]:
ttops = gpd.read_file("Python Files/Data/SB2_tree_crowns_segmented.shp")

### Clipping all lidar points which fall within the collective tree top area:

In [ ]:
gdf = gpd.GeoDataFrame(voxel_df, geometry=gpd.points_from_xy(voxel_df.X, voxel_df.Y))
points_clip = gpd.clip(gdf, ttops)

### Setting up necessary variables for clipping processes:

In [ ]:
trees = []
cpu_cores = 6

# Copy ttops, & convert to dask:
ttops_cpy = ttops.copy()
ttops_test_ddf = dask_gpd.from_geopandas(ttops_cpy, npartitions = cpu_cores)  # Adjust the number of partitions as needed

# copy points_clip & convert to dask:
points_clip_cpy = points_clip.copy()
points_clip_ddf = dask_gpd.from_geopandas(points_clip_cpy, npartitions = cpu_cores)  # Adjust the number of partitions as neededgeometry)  # Ensure 'Tree' is correctly defined in your code

unique_tree_ids = ttops_test_ddf["treeID"].unique().compute()

# sort by ascending (for recrusion)
ttops_cpy.sort_values(by=['treeID'], ascending = True)

# Set 'tree_id' as the index:
ttops_test_ddf = ttops_test_ddf.set_index('treeID')

# Traditional Clipping:

The provided code processes Lidar data for each tree in the ttops_cpy GeoDataFrame by clipping Lidar points against corresponding tree geometries. It iterates through the trees, clipping Lidar points using geopandas.clip, computing adjusted statistics, and appending the clipped Lidar geometries. The processing time is measured and displayed, and the resulting clipped Lidar geometries are combined and visualized in a plot.

In [ ]:
start_time = time.time()

ttops_cpy = ttops_cpy.sort_values(by=['treeID'], ascending = True)

lidar_only = []

with ProgressBar():
    for index, row in ttops_cpy.iterrows():
        tree_id = row["treeID"]
        clipped_lidar = gpd.clip(points_clip_cpy, row.geometry)
        clipped_lidar_geometry = clipped_lidar.geometry
        
        # LAI calculations (sensitive):
        # pop_tree_total, area = total_tree_adjusted(clipped_lidar, ttops_cpy) # returns pop_tree_total & area, modifies ttops.
        
        print(f'Tree ID: {tree_id}. Clipped points: {len(clipped_lidar_geometry)}')
        
        lidar_only.append(clipped_lidar_geometry)
        new_tree = Tree(tree_id, clipped_lidar_geometry)
        trees.append(new_tree)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds")

# Append each GeoSeries to the combined_geoseries
combined_geoseries = pd.concat(lidar_only, ignore_index=True)

# Lidar points clipped:
fig, ax = plt.subplots(figsize=(20,20))
combined_geoseries.plot(ax = ax, facecolor = "blue", alpha=0.5, markersize =1)
plt.show()

# Recursive Clipping:

The recursive_clipped_geometries function executes a recursive approach to compute clipped geometries for a specified set of tree geometries and points. The function takes parameters such as a set of points (GeoDataFrame), a list of tree IDs, a GeoDataFrame containing tree geometries, and an optional accumulator list for storing the clipped geometries. Within this function, the recursive process begins by examining the last tree ID in the provided list. It clips the points against the geometry of the corresponding tree and appends the resulting clipped geometries. The recursion continues until all tree IDs have been processed, ensuring that the points are correctly filtered and clipped for each tree geometry. The overall objective is to accumulate the clipped geometries for each tree and display the elapsed time and total clipped points at the end of the process.

In [ ]:
sys.setrecursionlimit(4000)

# Recursive function to compute clipped geometries
def recursive_clipped_geometries(points, tree_ids, ttops, accumulator=[]):
    if not tree_ids:
        return accumulator

    tree_id = tree_ids[-1]
    tree_row = ttops.iloc[tree_id]  # Get row, by index.

    # Clip the points against the tree_row.geometry
    tree_gdf = gpd.GeoDataFrame(geometry=[tree_row.geometry])
    clipped_points = gpd.clip(points, tree_gdf)

    print(f'Tree ID: {tree_id}. Clipped points: {len(clipped_points)}')

    # Append the clipped geometries to the accumulator
    accumulator.append(clipped_points)

    # Filter points to remove all rows corresponding to tree_id
    points = points[~points.index.isin(clipped_points.index)]

    # Recur with the remaining tree_ids and updated points
    return recursive_clipped_geometries(points, tree_ids[:-1], ttops, accumulator)



# Call the recursive function
start_time = time.time()
clipped_lidar = recursive_clipped_geometries(points_clip_cpy, list(range(len(ttops_cpy) - 1, -1, -1)), ttops_cpy)
end_time = time.time()

# Print elapsed time and number of resulting points
print(f"Elapsed time: {end_time - start_time} seconds")
print(f"Total Clipped Points: {sum(len(points) for points in clipped_lidar)}")

# Dask Parallel Clipping:

The dask_clipped_geometries function implements a spatial clipping operation using Dask for a specific tree geometry defined by its ID. It takes a tree ID and retrieves the corresponding tree geometry from the ttops_test_ddf DataFrame. The function then sets the coordinate reference system (CRS) for the provided Lidar data points to match the tree's CRS. Using Dask-based spatial join operations, the function clips the Lidar data points with the geometry of the specified tree based on the 'intersects' predicate. The resulting clipped geometries are returned as a GeoSeries. Additionally, the code organizes the computation by creating a list of delayed tasks for each tree ID, computes the clipped geometries in parallel, measures the elapsed time, and prints it along with other relevant information.

In [ ]:
def dask_clipped_geometries(tree_id):
    
    tree_row = ttops_test_ddf.loc[tree_id]  # Get row from index.
    
    points_clip_ddf.crs = tree_row.crs  # Coordinate reference system
    
    # Clipping equivalent using Dask.
    clipped_lidar = dask_gpd.sjoin(tree_row, points_clip_ddf, how="inner", predicate="intersects")
    
    return clipped_lidar.geometry



# Create a list of delayed tasks for each tree_id
delayed_tasks = [delayed(dask_clipped_geometries)(tree_id) for tree_id in unique_tree_ids]

start_time = time.time()

# Calling compute_clipped_geometries with delay and parallelism to enhance computation
with ProgressBar():
    print("Getting Geometries:")
    computed_geometries = compute(*delayed_tasks, scheduler="threads")
    
    print("Computing Dask Geometries:")
    computed_tables = [gdf.compute() for gdf in computed_geometries]

# Print Elapsed Time
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds")

# Plotting Random Tree from Clipped Trees:

In [ ]:
import random

random_tree = random.choice(trees)
rand_t_id = random_tree.tree_id
rand_t_clip = random_tree.lidar_points

# PLOTTING:
if len(rand_t_clip) > 0:
    fig, ax = plt.subplots(figsize=(10, 10))
    ttops.plot(ax=ax, facecolor="green", alpha=0.5)

    # Printing Debug:
    clipped_points = gpd.clip(points_clip, ttops[ttops["treeID"] == iD])

    rand_t_clip.plot(ax=ax, alpha=0.5, markersize=1)

    # Set the axis limits based on the extent of rand_t_clip
    rand_t_clip_total_bounds = rand_t_clip.total_bounds
    ax.set_xlim([rand_t_clip_total_bounds[0] - 250, rand_t_clip_total_bounds[2] + 500])
    ax.set_ylim([rand_t_clip_total_bounds[1] - 250, rand_t_clip_total_bounds[3] + 500])

    # Show the plot
    plt.show()